In [ ]:
epochs = 50

# भाग 7 - FederatedDataset के साथ Federated Learning

यहां हम फेडरेटेड डेटासेट का उपयोग करने के लिए एक नया टूल पेश करते हैं। हमने एक `FederatedDataset` वर्ग बनाया है, जिसका उपयोग PyTorch Dataset वर्ग की तरह करने के लिए किया जाता है, और एक फ़ेडरेटेड डेटा लोडर को दिया जाता है `FederatedDataLoader` जो एक फ़ेडरेटेड फ़ैशन में इसके लिए पुनरावृत्त करेगा।


लेखक:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Theo Ryffel - Github: [@LaRiffle](https://github.com/Laiffiff)

NbTranslate के उपयोग द्वारा अनुवाद किया गया:
Urvashi Bhattacharyya - Github: [@raheja](https://github.com/raheja)


हम सैंडबॉक्स का उपयोग करते हैं जो हमने पिछले पाठ की खोज की थी

In [ ]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

फिर एक डेटासेट की खोज करें

In [ ]:
boston_data = grid.search("#boston", "#data")
boston_target = grid.search("#boston", "#target")

हम एक मॉडल और एक अनुकूलक लोड करते हैं

In [ ]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1

model = th.nn.Linear(n_features, n_targets)

यहां हमने एक `FederatedDataset` में प्राप्त डेटा डाला। उन श्रमिकों को देखें जो डेटा का हिस्सा रखते हैं।

In [ ]:
# Cast the result in BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)
optimizers = {}
for worker in dataset.workers:
    optimizers[worker] = th.optim.Adam(params=model.parameters(),lr=1e-2)

हमने इसे `FederatedDataLoader` में रखा और विकल्प निर्दिष्ट किए

In [ ]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=32, shuffle=False, drop_last=False)

और अंत में हम युगों पर पुनरावृति करते हैं। आप देख सकते हैं कि शुद्ध और स्थानीय PyTorch प्रशिक्षण की तुलना में यह कैसे समान है!

In [ ]:
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        model.send(data.location)
        
        optimizer = optimizers[data.location.id]
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred.view(-1) - target)**2).mean()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 8 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBatch loss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))            
            
    print('Total loss', loss_accum)


# बधाई हो!!! - समुदाय में शामिल होने का समय!

इस नोटबुक ट्यूटोरियल को पूरा करने पर बधाई! यदि आपने इसका आनंद लिया है और एआई और एआई सप्लाई चेन (डेटा) के विकेन्द्रीकृत स्वामित्व के संरक्षण की ओर आंदोलन में शामिल होना चाहते हैं, तो आप निम्न तरीकों से ऐसा कर सकते हैं!

### GitHub पर स्टार PySyft

हमारे समुदाय की मदद करने का सबसे आसान तरीका सिर्फ रिपॉजिटरी को अभिनीत करना है! यह हमारे द्वारा बनाए जा रहे कूल टूल्स के बारे में जागरूकता बढ़ाने में मदद करता है।

- [स्टार PySyft](https://github.com/OpenMined/PySyft)

### हमारे Slack में शामिल हों!

नवीनतम प्रगति पर अद्यतित रहने का सबसे अच्छा तरीका हमारे समुदाय में शामिल होना है! [http://slack.openmined.org](http://slack.openmined.org) पर फॉर्म भरकर आप ऐसा कर सकते हैं

### एक कोड परियोजना में शामिल हों!

हमारे समुदाय में योगदान करने का सबसे अच्छा तरीका एक कोड योगदानकर्ता बनना है! किसी भी समय आप PySyft GitHub जारी करने वाले पृष्ठ पर जा सकते हैं और "Projects" के लिए फ़िल्टर कर सकते हैं। यह आपको सभी शीर्ष स्तर के टिकट दिखाएगा कि आप किन परियोजनाओं में शामिल हो सकते हैं! यदि आप किसी परियोजना में शामिल नहीं होना चाहते हैं, लेकिन आप थोड़ी सी कोडिंग करना चाहते हैं, तो आप "good first issue" चिह्नित गीथहब मुद्दों की खोज करके अधिक "one off" मिनी-प्रोजेक्ट्स की तलाश कर सकते हैं।

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### दान करना

यदि आपके पास हमारे कोडबेस में योगदान करने का समय नहीं है, लेकिन फिर भी समर्थन उधार देना चाहते हैं, तो आप हमारे ओपन कलेक्टिव में भी एक बैकर बन सकते हैं। सभी दान हमारी वेब होस्टिंग और अन्य सामुदायिक खर्च जैसे कि हैकाथॉन और मीटअप की ओर जाते हैं!

[OpenMined का ओपन कलेक्टिव पेज](https://opencollective.com/openmined)